# Wikipedia Text for Prodigy

1. Download Wikipedia Backup for your target language. e.g. `bnwiki-20190801-pages-articles-multistream.xml.bz2`
2. Extract the backup with WikiExtractor (https://github.com/attardi/wikiextractor) into the relative `text` dir.
3. Run this notebook.

In [ ]:
import json

import dask.dataframe as dd
import dask.bag as db

from dask.distributed import Client, progress

In [2]:
client = Client(threads_per_worker=1,
                n_workers=4,
                memory_limit='2GB')
client

Client Scheduler: tcp://127.0.0.1:43499 Dashboard: http://127.0.0.1:8787/status,Cluster Workers: 4 Cores: 4 Memory: 8.00 GB


### Load files

Load the extracted wikipedia JSON (https://github.com/attardi/wikiextractor)

In [7]:
import glob

files_paths = listing = glob.glob('text/*/*')

'Total {} files'.format(len(file_paths))

'Total 278 files'

### Setup Func for Splitting lines

`split_lines` takes a `dict` and processes it for the Prodigy JSON format.

In [8]:
import re

PATTERN = re.compile(r'\s*[\.\!\?।]\s*')

def split_lines(document):
    meta = {'source': '{} - {}'.format(document['title'], document['url'])}
    ret = []
    for line in PATTERN.split(document['text']):
        ret.append({'text': line, 'meta': meta})
    return ret
    

In [9]:
documents = db.read_text(file_paths) \
    .map(json.loads) \
    .map(split_lines) \
    .map(json.dumps)

In [ ]:
documents.to_textfiles('data/*.jsonl')